In [1]:
!pip install --upgrade transformers datasets accelerate peft

  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.0
    Uninstalling datasets-3.0.0:
      Successfully uninstalled datasets-3.0.0


In [2]:
# This step is needed only on Apple Metal
!pip uninstall bitsandbytes -y

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # Replace with the appropriate Llama 3.1 model name

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set up LoRA configuration
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout probability
    bias="none",  # Don't add bias to the LoRA adapters
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to specific attention layers
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# Check if pad token exists, add it if missing
if tokenizer.pad_token is None:
   print(f'Added pad_token {tokenizer.eos_token}')
   tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

# Update the model's token embeddings to accommodate the new pad token
model.resize_token_embeddings(len(tokenizer))

Added pad_token <|eot_id|>


Embedding(128256, 4096)

In [5]:
import torch
print(f'MPS available: {torch.backends.mps.is_available()}')

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = model.to(device)

MPS available: True


Prepare data

In [6]:
from datasets import load_dataset

dataset = load_dataset("ruslanmv/ai-medical-chatbot", split='all')
dataset

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 256916
})

In [7]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}


In [8]:
# Inspired by https://github.com/meta-llama/llama-recipes/blob/main/src/llama_recipes/datasets/alpaca_dataset.py
# See also https://crfm.stanford.edu/2023/03/13/alpaca.html

from copy import deepcopy

IGNORE_INDEX = -100

# Tokenize the dataset
def tokenize_function(example):
    if example.get("Patient", "") == "":
        prompt = PROMPT_DICT["prompt_no_input"].format(instruction=example['Description'])
    else:
        prompt = PROMPT_DICT["prompt_input"].format(instruction=example['Description'], input=example['Patient'])

    # In decoder-only transformers, the task is to predict the next token.
    # The sequence is fed as input_ids, and labels are the next token in the sequence.
    # Suppose the sequence is: ["The", "capital", "of", "France", "is", "Paris"]
    # Input (input_ids): ["The", "capital", "of", "France", "is", "Paris"]
    # Labels: ["capital", "of", "France", "is", "Paris", "<EOS>"] (Shifted version of the input)

    formatted_example = prompt + example['Doctor']

    encoded_prompt = torch.tensor(
            tokenizer.encode(prompt), dtype=torch.int64
        )
        
    example = tokenizer.encode(formatted_example)
    example.append(tokenizer.eos_token_id)
    example = torch.tensor(
        example, dtype=torch.int64
    )
    labels = deepcopy(example)
    labels[: len(encoded_prompt)] = -1
    example_mask = example.ge(0)
    label_mask = labels.ge(0)
    example[~example_mask] = 0
    labels[~label_mask] = IGNORE_INDEX  # Avoid calculating loss on prompt predictions

    return {
        "input_ids": example.tolist(),
        "labels": labels.tolist(),
        "attention_mask": example_mask.tolist(),
    }


tokenized_dataset = dataset.map(tokenize_function, remove_columns=['Description', 'Patient', 'Doctor'], batched=False)
tokenized_dataset

Map:   0%|          | 0/256916 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'labels', 'attention_mask'],
    num_rows: 256916
})

In [9]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(100))

Training

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding='longest',
)

In [11]:
import math

# Define the perplexity metric function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Shift the labels so that they're aligned with the next token prediction
    labels = labels[:, 1:].reshape(-1)
    logits = logits[:, :-1].reshape(-1, logits.shape[-1])
    
    # Compute cross-entropy loss
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(logits, labels)

    # Compute perplexity from loss
    perplexity = math.exp(loss.item()) if loss.item() < 100 else float("inf")
    
    return {"perplexity": perplexity}

In [12]:
from transformers import TrainingArguments, Trainer
from torch.optim import AdamW

training_args = TrainingArguments(
    output_dir="./llama_lora_finetuned",
    eval_strategy="steps",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    save_steps=1000,
    logging_steps=50,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=10,
    load_best_model_at_end=True,
)

optimizer = AdamW(model.parameters(), lr=5e-5)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    optimizers=(optimizer, None),  # No need for a scheduler here
    compute_metrics=compute_metrics
)


Without quantization, training fails on MBP with 64 GB RAM when computing metrics (perplexity). 
It works with batch size of 1 without metrics computation

In [13]:
trainer.train()

  0%|          | 0/51383 [00:00<?, ?it/s]

{'loss': 3.0558, 'grad_norm': 40.90482711791992, 'learning_rate': 4.996106904405038e-05, 'epoch': 0.0}


  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 72.76 GB, other allocations: 162.05 MB, max allowed: 81.60 GB). Tried to allocate 10.43 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
model.save_pretrained("./llama3_lora_model_finetuned")
tokenizer.save_pretrained("./llama3_lora_tokenizer_finetuned")